### α段階のモノ
- いっぺんに動かして問題がないかをチェックするもの
- 実験データを読み込んで半自動化できるかを試みる。

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from pandas import DataFrame

data = pd.read_excel(".xlsx",sheet_name=0,header = None)
df0 = data.drop(data.index[0:6])
df0_1 = df0.reset_index()
del df0_1["index"]
df1 = df0_1.rename(columns={0:"Range=",1:"10.000V",2:"1.59V"})
df1["トルク1"] = df1["10.000V"]*-25
df1["トルク2"] = df1["1.59V"]*-41.522 - 12.253
df1["トルク1"].plot()

t = [df1["トルク1"][u]-df1["トルク1"][u-1] for u in range(1,len(df1))]
t.insert(0,0)
df1["トルク1の増加率"] = t

df1_array = np.array([df1["トルク1"].to_list(),df1["トルク1の増加率"].to_list()])
df1_array = df1_array.T
result=KMeans(n_clusters=3).fit_predict(df1_array)
df1["result"] = result

for d in range(0,len(df1)):
    if df1["result"][d] != 0:
        break

for e in range(d,len(df1)):
    if df1["result"][e] == 0:
        break

for f in range(e,len(df1)):
    if df1["result"][f] != 0:
        break
        
df1["安静時平均"] = df1["トルク1"][0:d-1].mean() #安静時平均
df1["max"] = df1["トルク1"][0:e].max()
df1["トルク1"][0:d-1]

df1["MVC"] = df1["max"]  - df1["安静時平均"] 
df1["MVC*0.02"] = df1["MVC"] * 0.02
df1["offset"] = df1["トルク1"] - df1["安静時平均"]

i = 0
while True:
    if df1["MVC*0.02"][i] > df1["offset"][i]:
        i +=1
    else:
        print(df1["MVC*0.02"][i] > df1["offset"][i],i)
        break

k = df1["offset"][i+201] - df1["offset"][i]

df1["差"] = k
df1["RFD"] = k/0.2

df2 = df1[["トルク1","トルク1の増加率"]][e:]#と定義しなおす。
df2["安静時平均"] = df2["トルク1"][:f-1-e].mean() #安静時平均
df2["max"] = df2["トルク1"].max()
df2["MVC"] = df2["max"]  - df2["安静時平均"] 
df2["MVC*0.02"] = df2["MVC"] * 0.02
df2["offset"] = df2["トルク1"] - df2["安静時平均"]

i = e+2000
while True:
    if df2["MVC*0.02"][i] > df2["offset"][i]:
        i +=1
    else:
        print(df2["MVC*0.02"][i] > df2["offset"][i],i)
        break

k = df2["offset"][i+201] - df2["offset"][i]
df2["差"] = k
df2["RFD"] = k/0.2

with pd.ExcelWriter('1kHzの解析.xlsx') as writer:
    df1.to_excel(writer, sheet_name='一つ目の山')
    df2.to_excel(writer, sheet_name='二つ目の山')

False 8403
False 36105
